In [1]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
#import mpld3

In [2]:
print("Hello world!")

Hello world!


<H1>Import Wikipedia Data</H1>

In [3]:
data = pd.read_csv(r"C:\temp\output.csv")
data.head()

,Unnamed: 0,0
0,A,"#Type: regular article\r\nA (named , plural As..."
1,Albedo,"#Type: regular article\r\nAlbedo () (albedo, m..."
2,An American in Paris,#Type: regular article\r\nAn American in Paris...
3,Autism,#Type: regular article\r\n \r\nAutism is a dev...
4,Achilles,"#Type: regular article\r\nIn Greek mythology, ..."


In [4]:
titles  = data["Unnamed: 0"].tolist()
synopses = data["0"].tolist()

nr_articles = len(data)
titles = titles[:nr_articles]
synopses = synopses[:nr_articles]

In [5]:
#titles[999]
len(data)

13385

<H1>Stopword removal, Stemming and Tokenizing</H1>

In [6]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [7]:
#nltk.download()

In [8]:
# load nltk's English stopwords as variable called 'stopwords'
#stopWords = set(stopwords.words('english'))
stopwords = nltk.corpus.stopwords.words('english')
print (stopwords[:10])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [9]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [10]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [11]:
#not super pythonic, no, not at all.
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
cnt = 0
for i in synopses:
    if cnt%100 == 0 :
        print(cnt)
    cnt = cnt+1
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300


In [22]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print ( 'there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')


there are 45778663 items in vocab_frame


In [23]:
print(len(totalvocab_tokenized))
print(len(totalvocab_stemmed))

#totalvocab_stemmed.append("hacks")

45778663
45778663


In [24]:

print (vocab_frame.head())
print ()
print ()
print ()
print ()

           words
type        type
regular  regular
articl   article
a              a
name       named






<H1>TF-idf similarity</H1>

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(synopses) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

C:\Users\pieder\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Wall time: 1h 11min 17s
(13385, 693)


In [26]:
terms = tfidf_vectorizer.get_feature_names()

In [27]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)
print 
print

<function print>

In [28]:
print(dist)

[[-2.22044605e-16  9.48454596e-01  8.88030203e-01 ...  9.47561306e-01
   9.40035046e-01  8.75928156e-01]
 [ 9.48454596e-01  0.00000000e+00  9.14082370e-01 ...  8.83295380e-01
   8.77809365e-01  8.21998707e-01]
 [ 8.88030203e-01  9.14082370e-01  0.00000000e+00 ...  9.18644653e-01
   8.92124166e-01  7.86359177e-01]
 ...
 [ 9.47561306e-01  8.83295380e-01  9.18644653e-01 ...  1.11022302e-16
   8.73776046e-01  8.75348981e-01]
 [ 9.40035046e-01  8.77809365e-01  8.92124166e-01 ...  8.73776046e-01
   0.00000000e+00  7.68399875e-01]
 [ 8.75928156e-01  8.21998707e-01  7.86359177e-01 ...  8.75348981e-01
   7.68399875e-01 -2.22044605e-16]]


<H1>Clustering</H1>

In [39]:
from sklearn.cluster import KMeans

num_clusters = 200

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

Wall time: 24min 21s


In [40]:
from sklearn.externals import joblib

#uncomment the below to save your model 
#since I've already run my model I am loading from the pickle

joblib.dump(km,  'doc_cluster200.pkl')

km = joblib.load('doc_cluster200.pkl')
clusters = km.labels_.tolist()

In [41]:
ranks = list(range(nr_articles))
genres = list(range(nr_articles))
#ranks
print(len(titles))
print(len(ranks))
print(len(synopses))
print(len(genres))
print(len(clusters))
print(km)


13385
13385
13385
13385
13385
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=200, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)


In [42]:
films = { 'title': titles, 'rank': ranks, 'synopsis': synopses, 'cluster': clusters, 'genre': genres }

frame = pd.DataFrame(films, index = [clusters] , columns = ['rank', 'title', 'cluster', 'genre'])

In [43]:
frame['cluster'].value_counts() #number of films per cluster (clusters from 0 to 4)

64     330
78     246
53     217
5      188
11     182
63     178
176    143
142    142
117    133
131    132
106    130
120    127
16     124
93     122
138    120
90     120
10     119
104    119
179    118
71     116
9      115
129    114
85     113
45     113
149    111
32     109
28     107
58     105
22     105
17     105
      ... 
182     29
4       29
55      28
199     28
135     27
160     27
2       27
73      27
137     26
119     26
148     26
163     26
189     25
74      25
115     24
124     24
150     24
36      23
121     23
141     22
146     21
168     21
171     20
19      20
192     19
169     19
77      15
180     14
62      13
114     11
Name: cluster, Length: 200, dtype: int64

In [44]:
grouped = frame['rank'].groupby(frame['cluster']) #groupby cluster for aggregation purposes

grouped.mean() #average rank (1 to 100) per cluster

cluster
0       6804.294118
1       7268.921348
2       7313.444444
3       5929.500000
4       6829.448276
5       6761.813830
6       6530.805556
7       7643.901961
8       6295.606383
9       6314.660870
10      6869.697479
11      6769.802198
12      5713.317073
13      5513.466667
14      7328.049383
15      7755.666667
16      7155.620968
17      6962.038095
18      6463.216216
19      4006.750000
20      7242.822581
21      8102.405405
22      7491.838095
23      6622.327869
24      6273.047619
25      6634.458824
26      7204.046875
27      6208.220588
28      6957.934579
29      6400.210526
           ...     
170     4801.609756
171     8535.150000
172     6885.326733
173     8200.631579
174     6728.865854
175     6508.607143
176     7176.251748
177     6354.888889
178     7124.672727
179     6973.881356
180     7496.142857
181     6347.880000
182     6133.620690
183     6779.069767
184     6711.970149
185     5016.216216
186     6462.788732
187     7033.254237
188     7112

In [45]:
from __future__ import print_function

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    #print("Cluster %d words:" % i, end='')
    
    #for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
    #    print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    #print() #add whitespace
    #print() #add whitespace
    
    print("Cluster %d titles:" % i, end='')
    for title in frame.ix[i]['title'].values.tolist():
        print(' %s,' % title, end='')
    print() #add whitespace
    print() #add whitespace
    
print()
print()

Top terms per cluster:

Cluster 0 titles: Anthemius of Tralles, Analytic geometry, Brouwer fixed-point theorem, Boiling point, Bézier curve, Craps, Circle, Constructible number, Cartesian coordinate system, Delaunay triangulation, Diophantine equation, Ellipse, Euclidean geometry, Elliptic curve, Fahrenheit, Cis-trans isomerism, Geodesy, Hyperbola, Longitude, Latitude, Lagrangian point, Parabola, Polygon, Perimeter, Pythagorean triple, Projective plane, Pointing device, Polar coordinate system, Quadrilateral, Sphere, Stokes' theorem, Statite, Sierpinski carpet, Sierpinski triangle,

Cluster 1 titles: Foreign relations of Angola, Latin American Integration Association, Foreign relations of Brunei, Foreign relations of Brazil, Foreign relations of Burkina Faso, Foreign relations of Burundi, Brezhnev Doctrine, Country, Country code, Foreign relations of Cameroon, Foreign relations of the Central African Republic, Foreign relations of Chile, Foreign relations of Croatia, Foreign relations 

C:\Users\pieder\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


 Sysop, Sheffer stroke,

Cluster 22 titles: Telecommunications in Antigua and Barbuda, Telecommunications in Anguilla, Telecommunications in Belarus, Telecommunications in Botswana, Telecommunications in Bulgaria, Telecommunications in Burkina Faso, Communications in Burundi, BBC Radio 1, Telecommunications in Chad, Telecommunications in Cambodia, Telecommunications in Cameroon, Telecommunications in the Cayman Islands, Telecommunications in the Central African Republic, Telecommunications in Chile, Communications in Colombia, Telecommunications in the Czech Republic, Telecommunications in the Democratic Republic of the Congo, Telecommunications in Dominica, Telecommunications in Ecuador, Telecommunications in El Salvador, Telecommunications in Equatorial Guinea, Telecommunications in Eritrea, Telecommunications in Fiji, Telecommunications in the Faroe Islands, Telecommunications in Finland, Telecommunications in French Polynesia, Telecommunications in French Guiana, Telecommunications

Cluster 43 titles: Alexei Petrovich, Tsarevich of Russia, Aegeus, Aegisthus, Adoptionism, Benjamin, Emperor Shōmu, Emperor Kanmu, Emperor Jimmu, Marquess of Aberdeen and Temair, Emperor Yōmei, Emperor Bidatsu, Emperor Kinmei, Emperor Sushun, Emperor Kōbun, Empress Suiko, Emperor Kōtoku, Empress Kōgyoku, Emperor Tenji, Emperor Monmu, Empress Jitō, Empress Genshō, Emperor Junnin, Emperor Tenmu, Empress Kōken, Emperor Sukō, Emperor Heizei, Emperor Junna, Emperor Go-Kōgon, Emperor Annei, Emperor Suizei, Emperor Kōrei, Emperor Kōshō, Emperor Kōan, Emperor Kōgen, Emperor Kaika, Emperor Seimu, Emperor Keikō, Emperor Suinin, Emperor Chūai, Emperor Sujin, Emperor Nintoku, Emperor Ōjin, Emperor Hanzei, Emperor Richū, Emperor Ankō, Emperor Ingyō, Emperor Ankan, Emperor Buretsu, Emperor Seinei, Emperor Kenzō, Emperor Ninken, Emperor Senka, Emperor Keitai, Emperor Montoku, Emperor Ninmyō, Emperor Seiwa, Emperor Yōzei, Emperor Kōkō, Emperor Kazan, Emperor Uda, Filioque, Isaac, Julio-Claudian dynasty

Cluster 63 titles: Actrius, Andrei Tarkovsky, Alfred Hitchcock, American Film Institute, Akira Kurosawa, Alfonso Cuarón, Auguste and Louis Lumière, Army of Darkness, The Birth of a Nation, Blade Runner, Bille August, Blazing Saddles, Blue Velvet (film), Blade Runner 3: Replicant Night, Brian De Palma, Buffy the Vampire Slayer (film), Barry Lyndon, Bollywood, Braveheart, Batman (1966 film), Batman Returns, Batman (1989 film), Batman & Robin (film), Batman Forever, Bride of the Monster, Charlie Chaplin, Citizen Kane, Crouching Tiger, Hidden Dragon, Comedy film, Cult film, Chariots of Fire, Cecil B. DeMille, Chris Marker, Cahiers du cinéma, D. W. Griffith, David Cronenberg, David Lynch, David Fincher, Documentary film, Dressed to Kill (1980 film), Dr. Strangelove, Dziga Vertov, Das Boot, Dacoity, Dolores Fuller, The Evil Dead, Errol Morris, Spain in Flames, Eyes Wide Shut, Enter the Dragon, Evil Dead II, Ed Wood, Fox Film, Francis Ford Coppola, Cinema of Germany, Film editing, François Tr

...

Cluster 120 titles: Amsterdam, Aarhus, Adelaide, Athens, Acapulco, Aberdeen, Algiers, Ann Arbor, Michigan, Albion, Michigan, Alicante, Ajaccio, Alexandria, Romania, Atlanta, Bursa, Brussels, Buffalo, New York, Bristol City F.C., Barcelona, Baghdad, Bishkek, Baku, Brasília, Ballarat, Belfast, Copenhagen, City, Cardiff, Chernobyl, Cairo, City of London, Chicago, Casablanca, Carson City, Nevada, Delft, Darwin, Northern Territory, Dar es Salaam, Dublin, Denver, Davenport, Iowa, Detroit, Dundee, Damascus, Catan: Cities & Knights, Derry, Dimona, Eindhoven, Edinburgh, Edo, Fredericton, Fortaleza, Fort Wayne, Indiana, Florence, Guatemala City, Gothenburg, Ghent, Guangzhou, Glasgow City Chambers, Gotham City, Hobart, Helsinki, Houston, Hamilton, Ontario, Islamabad Capital Territory, Jerusalem, Jalalabad, Jakarta, Kabul, Kraków, Kuala Lumpur, Kagoshima, Kaohsiung, Kemerovo, Karachi, Kathmandu, Kandahar, Kansas City, Missouri, Lincoln, England, London, Lisbon, Liverpool, Los Angeles, Luanda, Mexi

Cluster 142 titles: Abdul Alhazred, Aaron, Afterlife, Abraham, Asgard, Álfheimr, Ask and Embla, Alcmene, Amalric of Bena, Amram, Anaxarchus, Anaxagoras, Andocides, Ānanda, Ark of the Covenant, Abba Arika, Abbahu, Ahimsa, Amphisbaena, Armageddon, Androphagi, Gautama Buddha, Bifröst, Baldr, Bodhidharma, Book of Proverbs, Book of Hosea, Book of Obadiah, Circe, Childe's Tomb, Cerberus, Callisto (mythology), Cuauhtémoc, Covenant-breaker, Death, Death metal, Draupnir, Draugr, Devil, Dragon, Dune Messiah, Donn, Danse Macabre, Dagome iudex, Enlil, Elijah, Ezekiel, Epitaph, Fenrir, Freyr, Freyja, Frigg, Forseti, Four Noble Truths, Father Christmas, Geri and Freki, Gospel of Barnabas, Gospel of Mark, Gospel of Matthew, Ghost, Book of Genesis, Goddess, Götterdämmerung, Garden of Eden, Gilgamesh, Hypnos, Huginn and Muninn, Heimdallr, Hel (being), Hero, Hadith, Heaven, Heracles, Hollow Earth, Höðr, Haddocks' Eyes, Horned God, Hengist and Horsa, Henotheism, Incest, Imhotep, Imbolc, Inquests in Engla

Cluster 157 titles: Abensberg, Anbar (town), Anah, Aalen, Abae, Aeclanum, Abertillery, Abergavenny, Accrington, Alexandria, West Dunbartonshire, Transport in the British Virgin Islands, Barcelonnette, Boone, North Carolina, Bodmin, Bamberg, Chojnów, Cape Town, County town, Cluny, Dartmouth, Devon, Dedham, Massachusetts, Dachau, Enniskillen, Emsworth, Fairmount, Indiana, Fort William, Highland, Guernica, Gilbert Plains, Gavoi, Hilter, Helsingborg Municipality, Hilversum, Hradčany, Hastings, Hanover, New Hampshire, Hultsfred Municipality, Hawick, Hatfield, Hertfordshire, Harwich, Harwich, Massachusetts, Hedeby, Hamar, Kingston upon Thames, Kajang, Kemp Town, Knaresborough Castle, Lincoln, New Hampshire, Longmeadow, Massachusetts, Limburg an der Lahn, Landau, Maser, Montauban, Montalcino, Merseburg, Oelde, People Against Gangsterism and Drugs, Posada, Sardinia, Padstow, Quedlinburg, Quantico, Virginia, Rungholt, Rosmalen, Stow-on-the-Wold, Sigtuna Municipality, Srebrenica, Scansano,

Clus

Cluster 172 titles: Ambiguity, Allah, Angst, Allophone, Affix, Australian English, Comparison of American and British English, Afghan, Abracadabra, British English, Basic English, Khmer language, Copula (linguistics), Cognate, Canadian English, English in the Commonwealth of Nations, Clitic, Collective noun, Rhyming slang, Clabbers, Dictionary, Dyslexia, Daoism-Taoism romanization issue, Dolmen, Daffynition, Longest word in English, Euphemism, Elf, English orthography, E-Prime, Esperanto grammar, West Flemish, Forth (programming language), Fart (word), False cognate, False friend, False etymology, Felsic, Gibberish, Great Vowel Shift, Grammatical conjugation, -gry puzzle, Modern Hebrew phonology, Hungarian language, Hiberno-English, Hoosier, Intension, International English, Infinitive, Jabberwocky, Kenning, Knout, Calligra Words, Loglan, Louchébem, Loch, Lexicography, Lexeme, Lexicology, Leet, Morpheme, Muslims, Midgard, Manner of articulation, Macaroni, Microsoft Word, Morphology (li

Cluster 197 titles: Atom, Acoustic theory, Alpha decay, Atomic theory, Angular momentum, Brownian motion, Baryon, Beta decay, Body mass index, Chandrasekhar limit, Concentration, Examples of differential equations, Density, Double planet, Energy, Enthalpy, Elementary particle, Googolplex, Galaxy groups and clusters, Giant planet, Global warming potential, Hadron, Kilogram, Kinetic energy, Law of definite proportions, Mass, Mass media, Molecular mass, Meson, Muon, Mole fraction, Mass transfer, Neutron, Nuclear physics, Neutrino, Nuclear fusion, Nucleon, Nuclear fission, Oscillation, Proton, Proteome, Proton decay, PCAA, Proton-proton chain reaction, Quark, Rutherford scattering, Simple harmonic motion, Spacecraft propulsion, Stoichiometry,

Cluster 198 titles: Aegean civilization, Ajanta Caves, Archaeoastronomy, Bronze Age, Broch, Çatalhöyük, Crannog, Chambered cairn, Chalcolithic, Epipalaeolithic Near East, Geocaching, Shock site, Hot or Not, Harappa, Iron Age, Kenyanthropus, Kremvax, 